Twitter API Demo

#### Importar librerias

In [ ]:
!pip install tweepy
!pip install dnspython
!pip install pymongo[srv]
!pip install pymongo[tls]
!pip install plotly
!pip install pandas

# !!!Restart runtime for changes to take effect!!! #

     |████████████████████████████████| 241 kB 4.1 MB/s 


#### Funciones


API Keys

In [ ]:
consumer_key = '###'
consumer_secret = '###'
access_token = '###'
access_secret = '###'

API Access

In [ ]:
import tweepy

# setup auth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# api
api = tweepy.API(auth)

API-related Funcs

In [ ]:
from datetime import datetime
import pandas as pd

def followers_date(username):
    today = datetime.now().date()
    today = str(today)

    user = api.get_user(username)
    followers_count = user.followers_count
    foll_dict = {'date': today, 'user': str.lower(user.screen_name), 'follows':
                 followers_count}
    insert_one_dbtwf(user.screen_name, today, foll_dict)

def statuses(username):
    content = api.user_timeline(screen_name=username, count=50, 
                                include_rts=False)

    for object in content:
        dict_obj = {'id': object.id, 'user': str.lower(object.user.screen_name),
                    'like': object.favorite_count, 'rt': object.retweet_count, 
                    'date': object.created_at}
        insert_one_dbtw(object.id, dict_obj)

def organize_follows_to_list(user):
  dictfol = find_follow(user)
  ls = list(map(lambda x : x['follows'], dictfol))
  lsdate = list(map(lambda x : x['date'], dictfol))

  df = pd.DataFrame(data={'date': lsdate, 'followers': ls})
  df = df.sort_values(by="date")

  line_series(df['date'], df, user, 'followers')

def organize_like_rt_to_list(user):
  dicttwe = find_db_tweet(x)
  lsl = list(map(lambda x : x['like'], dicttwe))
  lsr = list(map(lambda x : x['rt'], dicttwe))
  lsdate = list(map(lambda x : x['date'], dicttwe))

  df = pd.DataFrame(data={'date': lsdate, 'likes': lsl, 'rts': lsr})
  df = df.sort_values(by="date")
  line_series(df['date'], df, user, 'likes', 'rts')

def get_top_content(user):
  dicttwe = find_db_tweet(x)
  lsl = list(map(lambda x : x['like'], dicttwe))
  lsr = list(map(lambda x : x['rt'], dicttwe))
  lsdate = list(map(lambda x : x['date'], dicttwe))
  df = pd.DataFrame(data={'date': lsdate, 'likes': lsl, 'rts': lsr})
  df = df.sort_values(by="likes")
  print(df.head())




DB Connection

In [ ]:
from pymongo import MongoClient

client = MongoClient("###")

db = client.get_database('database')


DB Funcs

In [ ]:
# Tweets
def insert_one_dbtw(ID, item):
    records = db.twitter_records
    records.replace_one({'id': ID}, item, True)

# Followers
def insert_one_dbtwf(ID, date, item):
    records = db.twitter_follow
    records.replace_one({'user': ID, 'date':date}, item, True)

# Unused
def insert_multi_db(records, items):
    records.insert_many(items)
    client.close()

# Retrieve tweet, condition user
def find_db_tweet(condition):
    records = db.twitter_records
    return(list(records.find({'user':condition})))

# Retrieve all tweets # Unused
def show_all_tweet():
    records = db.twitter_records
    return(list(records.find()))

# Retrieve followcount, condition user
def find_follow(condition):
    records = db.twitter_follow
    return(list(records.find({'user':condition})))

# Retrieve all followcounts # Unused
def show_all_follow():
    records = db.twitter_follow
    return(list(records.find()))

#### Dashboard stuff

Plotly

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

def line_series(time, df, user, *argv):
  fig = make_subplots(rows=1, cols=1)
  title = '{0}: '.format(user)
  for arg in argv:
        title = '{0}{1} '.format(title, arg)
        fig.append_trace(go.Scatter(
            x=time,
            y=df[arg],
            name=arg),
            row=1,
            col=1
        )
  
  fig.update_layout(
        title_text=(title),
        title_x=0.5,
        font=dict(size=18))
  fig.show()


#### Main Loop


In [ ]:
x = input()
x = str.lower(x)


AlvaroUribeVel


In [ ]:
statuses(x)

In [ ]:
followers_date(x)

In [ ]:
organize_follows_to_list(x)


In [ ]:
organize_like_rt_to_list(x)

In [ ]:
get_top_content(x)

                  date  likes  rts
2  2021-11-19 00:18:51    243   68
14 2021-11-17 01:07:32    244   83
6  2021-11-18 01:18:34    358  110
0  2021-11-19 12:03:42    424  194
19 2021-11-16 00:23:40    657  231
